In [1]:
import numpy as np
import pandas as pd

########################################part 1#####################################
#erzeugen der Prognosefehler

#  Profil für Anzahl der Zeitschritte
df_profil = pd.read_excel("profil_winter.xlsx")

# Anzahl der Zufallsvariablen pro Zeitpunkt
num_variables = 32

# Anzahl der Szenarien
num_scenarios = 100

# Standardabweichung der normalverteilten Zufallsvariablen
std_dev = 0.005

# Erstelle ein leeres Pandas-DataFrame für die Ergebnisse
results = pd.DataFrame()

# Für jedes Szenario:   
for i in range(num_scenarios):
    # Erstelle ein Array mit normalverteilten Zufallsvariablen für jeden Bus und jeden Zeitpunkt
    random_variables = np.random.normal(0, std_dev, (len(df_profil), num_variables))
    
    # Berechne die Summe der Zufallsvariablen für jeden Zeitpunkt
    sum_random_variables = np.sum(random_variables, axis=1)
       
    # Berechne den Faktor, um den jede Zufallsvariable angepasst werden muss, damit die Summe 0 ergibt
    factor = -sum_random_variables / num_variables

    # Passe jede Zufallsvariable an, indem der Faktor multipliziert wird
    random_variables_adjusted = random_variables + factor.reshape(-1, 1)

    # Die gewünschte Toleranz weil die Berechnung von Gleitkommazahlen ungenau ist und somit die Summe der angepassten Zufallszahlen minimal von 0 abweichen kann
    tolerance = 1e-10

    # Prüfe, ob die Summe der angepassten Zufallsvariablen für jeden Zeitpunkt innerhalb der Toleranz ist
    assert np.allclose(np.sum(random_variables_adjusted, axis=1), 0, atol=tolerance)
    
    # Füge die Zufallsvariablen ein
    df_varied = pd.DataFrame(random_variables_adjusted)

    # Füge eine Spalte für den Zeitpunkt hinzu
    df_varied.insert(0, "t", range(1, len(df_profil)+1))

    # Setze die Spaltenüberschriften auf "S" + Nummer des Szenarios
    #df_varied.columns = ["S{}".format(i+1) for i in range(num_variables)]

    # Füge das DataFrame mit den angepassten Zufallsvariablen dem DataFrame der Ergebnisse hinzu
    results = pd.concat([results, df_varied], ignore_index=True)

# Füge eine Spalte für die Szenarien hinzu
results.insert(0, "Scenario", np.repeat(np.arange(1, num_scenarios+1), len(df_profil)))

# Sortiere die Ergebnisse nach Szenario und Zeitpunkt
results = results.sort_values(by=["Scenario", "t"])

# Speichere das DataFrame der Ergebnisse in einer CSV-Datei
results.to_excel("Ergebnisse.xlsx", index=False)



########################################part 2#####################################
load_bus = pd.read_excel('load_bus_winter.xlsx')

load_profil = pd.DataFrame(columns=['b'] + ['t'+str(i) for i in range(1,97)])
load_profil['b'] = load_bus['b']
for i in range(1,97):
    load_profil['t'+str(i)] = df_profil['profil'][i-1] * load_bus['Active demand']
    
print(load_profil)

     b        t1      t2        t3        t4        t5        t6        t7  \
0    1  0.000000  0.0000  0.000000  0.000000  0.000000  0.000000  0.000000   
1    2  0.070463  0.0700  0.068942  0.068633  0.068266  0.068180  0.068280   
2    3  0.063417  0.0630  0.062048  0.061769  0.061440  0.061362  0.061452   
3    4  0.084556  0.0840  0.082731  0.082359  0.081920  0.081816  0.081936   
4    5  0.042278  0.0420  0.041365  0.041180  0.040960  0.040908  0.040968   
5    6  0.042278  0.0420  0.041365  0.041180  0.040960  0.040908  0.040968   
6    7  0.140926  0.1400  0.137884  0.137265  0.136533  0.136361  0.136560   
7    8  0.140926  0.1400  0.137884  0.137265  0.136533  0.136361  0.136560   
8    9  0.042278  0.0420  0.041365  0.041180  0.040960  0.040908  0.040968   
9   10  0.042278  0.0420  0.041365  0.041180  0.040960  0.040908  0.040968   
10  11  0.031708  0.0315  0.031024  0.030885  0.030720  0.030681  0.030726   
11  12  0.042278  0.0420  0.041365  0.041180  0.040960  0.040908